In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress
import numpy as np

# Study data files
Top_Res_Path = "Data/WorldsBestRestaurants.csv"

# Read the Res data and the study results
Top_Res_df = pd.read_csv(Top_Res_Path)
Top_Res_df

,year,rank,restaurant,location,country,lat,lng
0,2002,1,El Bulli,Roses,Spain,42.263949,3.179553
1,2002,2,Restaurant Gordon Ramsay,London,United Kingdom,51.507218,-0.127586
2,2002,3,The French Laundry,Yountville,United States,38.401578,-122.360810
3,2002,4,Rockpool,Sydney,Australia,-33.868820,151.209295
4,2002,5,Spoon des Iles,Ile Maurice,Mauritius,-20.348404,57.552152
...,...,...,...,...,...,...,...
1045,2023,46,Orfali Bros Bistro,Dubai,United Arab Emirates,25.204849,55.270783
1046,2023,47,Mayta,Lima,Peru,-12.046689,-77.043088
1047,2023,48,La Grenouillère,La Madelaine-sous-Montreuil,France,50.466809,1.748056
1048,2023,49,Rosetta,Mexico City,Mexico,19.432608,-99.133208


In [6]:
# Checking the number of Restaurants.
Restaurant_count = Top_Res_df ['restaurant'].nunique()
Restaurant_count

257

In [7]:
# Checking the number of Countries.
Country_count = Top_Res_df ['country'].nunique()
Country_count

42

In [8]:
# Checking the number of Location.
Location_count = Top_Res_df ['location'].nunique()
Location_count

135